## Imports and Config

In [5]:
!pip install ccxt
!pip install fredapi

In [41]:
import ccxt
from fredapi import Fred

import numpy as np
import pandas as pd

import pickle
from datetime import datetime, timedelta

from sklearn.decomposition import PCA

## Load CSV File

In [ ]:
# dateparse = lambda dates: [datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dates]

In [10]:
df = pd.read_csv('IS5006_Historical.csv', index_col='datetime', parse_dates=[0], dayfirst=True)
df.head()

,Fuzzy_sentiment_signal,MA_Signal,inflation,unemployment,oil,gold_vol
datetime,,,,,,
2021-01-01,0,0,1.233584,6.4,48.13,21.0575
2021-01-02,0,0,1.233584,6.4,47.91,21.3250
2021-01-03,0,0,1.233584,6.4,47.69,21.5925
2021-01-04,0,0,1.233584,6.4,47.47,21.8600
2021-01-05,0,0,1.233584,6.4,49.78,22.1400


## Get Historical Prices

In [ ]:
exch = 'hitbtc'
period = '1d'
start = datetime(2020, 10, 31) # Earlier date to facilitate moving average calculation
end = datetime(2022, 3, 1)
limit = 1000
days = (end - start).days
symbol = 'BTC/USDT'

In [ ]:
historical_df = pd.DataFrame(columns=['datetime', 'Open', 'High', 'Low', 'Close', 'Volume'])
exchange = getattr(ccxt, exch)()
exchange.load_markets()
for i in range(int(days//limit)+1):
  since = int((start+timedelta(limit*i)).timestamp())*1000
  history = exchange.fetch_ohlcv(symbol, period, since, limit)
  temp_df = pd.DataFrame(history)
  temp_df.columns = ['datetime', 'Open', 'High', 'Low', 'Close', 'Volume']
  temp_df['datetime'] = pd.to_datetime(temp_df['datetime'], unit='ms')
  historical_df = historical_df.append(temp_df)
historical_df.head()

,datetime,Open,High,Low,Close,Volume
0,2020-10-31,13555.19,14101.91,13420.00,13798.99,26926.29303
1,2020-11-01,13785.26,13889.95,13617.36,13757.07,21207.00752
2,2020-11-02,13758.98,13833.38,13205.48,13550.57,22176.92171
3,2020-11-03,13551.75,14069.41,13281.98,14021.61,19809.05179
4,2020-11-04,14023.11,14259.03,13527.50,14141.88,30407.10620


## Add MA Crossover

In [ ]:
ma_df = historical_df.copy(deep = True)

In [ ]:
ma_df['EMA_10'] = ma_df['Close'].ewm(span=10, adjust = False).mean()
ma_df['SMA_50'] = ma_df['Close'].rolling(50).mean()
ma_df[f'MA_Position'] = np.where(ma_df[f'EMA_10'] > ma_df[f'SMA_50'], 1.0, 0.0)
ma_df[f'MA_Signal'] = ma_df[f'MA_Position'].diff()
ma_df.drop([f'MA_Position'], axis=1, inplace=True)
ma_df.set_index('datetime', inplace=True)
ma_df.index = pd.to_datetime(ma_df.index, errors='coerce')
ma_df.head()

,Open,High,Low,Close,Volume,EMA_10,SMA_50,MA_Signal
datetime,,,,,,,,
2020-10-31,13555.19,14101.91,13420.00,13798.99,26926.29303,13798.990000,NaN,NaN
2020-11-01,13785.26,13889.95,13617.36,13757.07,21207.00752,13791.368182,NaN,0.0
2020-11-02,13758.98,13833.38,13205.48,13550.57,22176.92171,13747.586694,NaN,0.0
2020-11-03,13551.75,14069.41,13281.98,14021.61,19809.05179,13797.409113,NaN,0.0
2020-11-04,14023.11,14259.03,13527.50,14141.88,30407.10620,13860.040184,NaN,0.0


In [ ]:
updated_df = df.merge(ma_df[['MA_Signal']], left_on=df.index, right_on=ma_df[['MA_Signal']].index, how='outer')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,fuzzy_2,fuzzy_2_signal,MA_Signal
datetime,,,
2020-10-31,NaN,NaN,NaN
2020-11-01,NaN,NaN,0.0
2020-11-02,NaN,NaN,0.0
2020-11-03,NaN,NaN,0.0
2020-11-04,NaN,NaN,0.0


In [ ]:
updated_df.to_csv('IS5006_Historical.csv')

## Add Macroeconomic Data

In [54]:
macro_series = {'inflation': 'FPCPITOTLZGUSA', 'unemployment': 'UNRATE', 
        'oil': 'DCOILWTICO', 'gold_vix': 'GVZCLS', 'cboe_vix':'VIXCLS', 'dow_jones': 'DJIA',
        'interest_rate': 'IR3TIB01USM156N'}

In [55]:
fred = Fred(api_key='40ed0248dfecd2195c2bc8353707eead')

In [56]:
macro_df = pd.DataFrame(index=df.index)
for key in macro_series.keys():
  temp_df = fred.get_series(macro_series[key])
  temp_df.name = key
  macro_df = macro_df.merge(temp_df, left_on=macro_df.index, right_on=temp_df.index, how='outer')
  macro_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
  macro_df.set_index('datetime', inplace=True)
  macro_df.sort_index(ascending=True, inplace=True)

In [57]:
macro_df.tail()

,inflation,unemployment,oil,gold_vix,cboe_vix,dow_jones,interest_rate
datetime,,,,,,,
2022-03-14,NaN,NaN,103.22,24.08,31.77,32945.24,NaN
2022-03-15,NaN,NaN,NaN,23.80,29.83,33544.34,NaN
2022-03-16,NaN,NaN,NaN,22.51,26.67,34063.10,NaN
2022-03-17,NaN,NaN,NaN,22.23,25.67,34480.76,NaN
2022-03-18,NaN,NaN,NaN,NaN,NaN,34754.93,NaN


In [58]:
macro_df['inflation'] = macro_df['inflation'].fillna(method="ffill")
macro_df['unemployment'] = macro_df['unemployment'].fillna(method="ffill")
macro_df['oil'] = macro_df['oil'].interpolate(method='time')
macro_df['gold_vix'] = macro_df['gold_vix'].interpolate(method='time')
macro_df['cboe_vix'] = macro_df['cboe_vix'].interpolate(method='time')
macro_df['dow_jones'] = macro_df['dow_jones'].interpolate(method='time')
macro_df['interest_rate'] = macro_df['interest_rate'].fillna(method="ffill")
macro_df.tail()

,inflation,unemployment,oil,gold_vix,cboe_vix,dow_jones,interest_rate
datetime,,,,,,,
2022-03-14,1.233584,3.8,103.22,24.08,31.77,32945.24,0.38
2022-03-15,1.233584,3.8,103.22,23.80,29.83,33544.34,0.38
2022-03-16,1.233584,3.8,103.22,22.51,26.67,34063.10,0.38
2022-03-17,1.233584,3.8,103.22,22.23,25.67,34480.76,0.38
2022-03-18,1.233584,3.8,103.22,22.23,25.67,34754.93,0.38


In [33]:
# df.drop(['inflation',	'unemployment',	'oil',	'gold_vol'], axis=1, inplace=True)

In [66]:
updated_df = df.merge(macro_df, left_on=df.index, right_on=macro_df.index, how='left')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
# updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MA_Signal,inflation,unemployment,oil,gold_vix,cboe_vix,dow_jones,interest_rate
datetime,,,,,,,,,
2021-01-01,0,0,1.233584,6.4,48.13,21.0575,23.805,30510.8325,0.14
2021-01-02,0,0,1.233584,6.4,47.91,21.3250,24.860,30415.1850,0.14
2021-01-03,0,0,1.233584,6.4,47.69,21.5925,25.915,30319.5375,0.14
2021-01-04,0,0,1.233584,6.4,47.47,21.8600,26.970,30223.8900,0.14
2021-01-05,0,0,1.233584,6.4,49.78,22.1400,25.340,30391.6000,0.14


In [67]:
# pca = PCA(n_components=3)
# pca.fit(updated_df[list(macro_series.keys())])
# print(pca.explained_variance_ratio_)
# with open('macro_pca.pkl','wb') as f:
#     pickle.dump(pca,f)

In [68]:
pca = None
with open('macro_pca.pkl', 'rb') as f:
    pca = pickle.load(f)

In [69]:
pca_cols = pd.DataFrame(pca.transform(updated_df[list(macro_series.keys())]), columns = [f'MACRO_{i}' for i in range(pca.n_components)], index=updated_df.index)
pca_cols.head()

,MACRO_0,MACRO_1,MACRO_2
datetime,,,
2021-01-01,3661.709111,-5.580458,0.925199
2021-01-02,3757.358041,-5.094828,1.771221
2021-01-03,3853.006971,-4.609197,2.617242
2021-01-04,3948.655902,-4.123567,3.463264
2021-01-05,3780.935518,-3.040439,1.718721


In [70]:
updated_df.drop(list(macro_series.keys()), axis=1, inplace=True)
updated_df = updated_df.merge(pca_cols, left_on=updated_df.index, right_on=pca_cols.index)
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MA_Signal,MACRO_0,MACRO_1,MACRO_2
datetime,,,,,
2021-01-01,0,0,3661.709111,-5.580458,0.925199
2021-01-02,0,0,3757.358041,-5.094828,1.771221
2021-01-03,0,0,3853.006971,-4.609197,2.617242
2021-01-04,0,0,3948.655902,-4.123567,3.463264
2021-01-05,0,0,3780.935518,-3.040439,1.718721


In [71]:
updated_df.tail()

,Fuzzy_sentiment_signal,MA_Signal,MACRO_0,MACRO_1,MACRO_2
datetime,,,,,
2022-02-20,1,-1,334.488761,24.074482,0.790679
2022-02-21,0,0,455.129095,24.827410,0.645460
2022-02-22,0,0,575.769429,25.580338,0.500242
2022-02-23,0,0,1040.616450,28.084669,1.338296
2022-02-24,0,0,948.544133,28.137896,0.788475


In [72]:
updated_df.index = pd.to_datetime(updated_df.index)
updated_df.to_csv('IS5006_Historical.csv')